In [1]:
import numpy as np
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from tqdm import tqdm
import tensorflow as tf
import tensorflow_datasets as tfds
from gensim.models import KeyedVectors

In [2]:
df_train = pd.read_csv("datasets/amazon_train.csv").fillna("")
NUM_DOCS = df_train.shape[0]
MAX_SENT = 660
MAX_WORDS = 1981
wv = KeyedVectors.load("embedding/amazon_embedding.wv")

In [3]:
def token_to_index(token, vocab, oov_token="UNK"):
    try:
        return vocab[token].index
    except KeyError:
        return vocab[oov_token].index

In [4]:
data = np.zeros(shape=(NUM_DOCS, MAX_SENT, MAX_WORDS), dtype=np.int32)
for i, doc in tqdm(enumerate(df_train.text)):
    sentences = sent_tokenize(doc)
    for j, sentence in enumerate(sentences):
        words = word_tokenize(sentence)
        for k, word in enumerate(words):
            data[i, j, k] = token_to_index(word, wv.vocab)

0it [00:00, ?it/s]


TypeError: int() argument must be a string, a bytes-like object or a number, not 'Vocab'